In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Kidney_Papillary_Cell_Carcinoma/GSE36133'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from the Cancer Cell Line Encyclopedia (CCLE)"
!Series_summary	"The Cancer Cell Line Encyclopedia (CCLE) project is a collaboration between the Broad Institute, the Novartis Institutes for Biomedical Research and the Genomics Novartis Foundation to conduct a detailed genetic and pharmacologic characterization of a large panel of human cancer models"
!Series_summary	"It consists of a compilation of gene expression, chromosomal copy number, and massively parallel sequencing data from nearly 1,000 human cancer cell lines. All raw and processed data are available through an integrated portal on www.broadinstitute.org/ccle"
!Series_overall_design	"The final cell line collection spans 36 cancer types. Representation of cell lines for each cancer type was mainly driven by cancer mortality in the United States, as a surrogate of unmet medical need, as well as availability."
Sample Characteristics Dictionary:
{0: ['primary site: central_ner

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 2
age_row = None
gender_row = None  # set to different values when applicable

# Define the conversion function for trait
def convert_trait(value):
    if isinstance(value, str):
        parts = value.split(':')
        if len(parts) > 1:
            key, val = parts[0].strip(), parts[1].strip().lower()
            if "clear_cell_renal_cell_carcinoma" in val or "renal_cell_carcinoma" in val:
                return 1  # Positive indicator for trait
            elif "papillary_carcinoma" in val:
                return 0  # Negative indicator or control
            else:
                return None
        else:
            return None
    else:
        return None

# Define the conversion function for age
def convert_age(value):
    try:
        if isinstance(value, str):
            parts = value.split(':')
            if len(parts) > 1:
                return float(parts[1].strip())
            else:
                return None
        elif isinstance(value, (int, float)):
            return float(value)
        else:
            return None
    except ValueError:
        return None

# Define the conversion function for gender
def convert_gender(value):
    if isinstance(value, str):
        parts = value.split(':')
        if len(parts) > 1:
            gender = parts[1].strip().lower()
            if gender == 'male':
                return 1
            elif gender == 'female':
                return 0
            else:
                return None
        else:
            return None
    elif isinstance(value, (int, float)):
        return int(value)
    else:
        return None

# Saving cohort information
save_cohort_info('GSE36133', './preprocessed/Kidney_Papillary_Cell_Carcinoma/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    # Clinical feature extraction
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Kidney_Papillary_Cell_Carcinoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/trait_data/GSE36133.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM886835': [None], 'GSM886836': [None], 'GSM886837': [None], 'GSM886838': [None], 'GSM886839': [None], 'GSM886840': [None], 'GSM886841': [None], 'GSM886842': [None], 'GSM886843': [None], 'GSM886844': [1], 'GSM886845': [1], 'GSM886846': [None], 'GSM886847': [None], 'GSM886848': [None], 'GSM886849': [None], 'GSM886850': [None], 'GSM886851': [None], 'GSM886852': [None], 'GSM886853': [None], 'GSM886854': [None], 'GSM886855': [None], 'GSM886856': [1], 'GSM886857': [None], 'GSM886858': [None], 'GSM886859': [None], 'GSM886860': [1], 'GSM886861': [None], 'GSM886862': [None], 'GSM886863': [1], 'GSM886864': [None], 'GSM886865': [None], 'GSM886866': [None], 'GSM886867': [None], 'GSM886868': [None], 'GSM886869': [None], 'GSM886870': [None], 'GSM886871': [None], 'GSM886872': [None], 'GSM886873': [None], 'GSM886874': [None], 'GSM886875': [0], 'GSM886876': [None], 'GSM886877': [None], 'GSM886878': [None], 'GSM886879': [None], 'GSM886880': [None], 'GSM886881': [None], 'GSM886882': [None], 'GSM88688

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['100009676_at', '10000_at', '10001_at', '10002_at', '10003_at',
       '100048912_at', '100049716_at', '10004_at', '10005_at', '10006_at',
       '10007_at', '10008_at', '100093630_at', '10009_at', '1000_at',
       '100101467_at', '100101938_at', '10010_at', '100113407_at', '10011_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'ORF': ['1', '10', '100', '1000', '10000'], 'Description': ['alpha-1-B glycoprotein', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'adenosine deaminase', 'cadherin 2, type 1, N-cadherin (neuronal)', 'v-akt murine thymoma viral oncogene homolog 3 (protein kinase B, gamma)']}


### Step 6: Gene Identifier Mapping

In [7]:
if requires_gene_mapping:
    # 1. Read the dictionary and decide the keys
    identifier_key = 'ID'
    gene_symbol_key = 'Description'

    # 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

    # 3. Apply the mapping with the 'apply_gene_mapping' function, and name the resulting gene expression dataframe "gene_data"
    gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE36133.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Kidney_Papillary_Cell_Carcinoma')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE36133', './preprocessed/Kidney_Papillary_Cell_Carcinoma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/GSE36133.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Kidney_Papillary_Cell_Carcinoma', the least common label is '0.0' with 1 occurrences. This represents 6.25% of the dataset.
The distribution of the feature 'Kidney_Papillary_Cell_Carcinoma' in this dataset is severely biased.

